In [113]:
import numpy as np
import pandas as pd

In [114]:
########## GRIEWANK #############
def bb_GRIEWANK(x):
    f = 1e20;
    f = 1.0 + (x[0]*x[0] + x[1]*x[1]) / 4000.0 - np.cos(x[0]) * np.cos(x[1]/np.sqrt(2.0))
    return f

startPtsGRIEWANK =np.array([[0.0,-357.4],
                    [535.1, 89.17],
                    [50.66, 324.4],
                    [160.9, 573.6],
                    [-386.4, 378.7],
                    [247.7, -189.6],
                    [-263.4, -159.3],
                    [-478.9, -269.4],
                    [-66.58, -479.8],
                    [287.7, -398.7],
                    [128.3, 13.74],
                    [91.78, 218.6],
                    [-423.1, -309.5],
                    [561.8, 493.1],
                    [208.3, -25.98],
                    [-181.8, -224.8],
                    [-554.8, -79.05],
                    [-105.3, -113.4],
                    [-27.35, 295.1],
                    [-340.2, 247.3],
                    [404.9, -540.3],
                    [-305.5, -504.9],
                    [373, 41.45],
                    [-507.4, -422.3],
                    [485.5, 128.7],
                    [335.3, 409.6],
                    [-595.2, 165.5],
                    [-232.8, -596.7],
                    [449.7, 458],
                    [-141.8, 521.2]])

In [115]:
####### COORDINATE SEARCH ALGORITHM - NO EXTENSIONS #######

"""
bb: Black box function, takes x0 as argument
x0: Tuple, inital point
step: Initial step distance
evalBdgt: Total evaluations budget 
upBound: Upper boundary for all dimension of x0
lowBound: Lower boundary for all dimension of x0
"""   
def coodSearch(bb, x0, step, evalBdgt, upBound, lowBound):
 
    initialEvBdgt = evalBdgt
    
    def checkOutOfBounds(x):
        return (x>upBound).sum()>0 or (x<lowBound).sum()>0
    
    if checkOutOfBounds(np.array(x0)):
        print("Initial points should be inside boundaries")
        return
    
    df = pd.DataFrame()
    cache = dict()
    minObj = bb(x0)
    cache[x0] = minObj
    evalBdgt = evalBdgt - 1
    
    #Main loop
    while(evalBdgt > 0):
        
        #Generating candidates
        points = np.zeros((len(x0)*2, len(x0)))
        values = np.full(len(x0)*2, np.inf)
        
        for k in range(len(x0)):
            points[k] = x0
            points[k,k] = points[k,k]+step
            points[len(x0)+k] = x0
            points[len(x0)+k, k] = points[len(x0)+k, k]-step
        
        #Evaluating candidates
        for i in range(len(points)):
            if(evalBdgt > 0):
                #failure case
                if(tuple(points[i]) in cache or checkOutOfBounds(points[i])):
                    step = step/2
                    break
                else:
                    values[i] = bb(points[i])
                    evalBdgt = evalBdgt-1
                    cache[tuple(points[i])] = values[i]
            else:
                break
        
        #Results
        minVal = np.min(values)
        
        if(minVal < minObj):
            #Updating final values
            minObj = minVal
            x0 = points[np.argmin(values)]
            
            #Creating new row for dataframe
            newRow = dict()
            newRow['Step'] = initialEvBdgt-evalBdgt
            for i in range(len(x0)):
                newRow[i]=x0[i]
            newRow['Obj'] = minObj
            df = df.append(newRow, ignore_index=True)
        
        #Stopping criterion
        if(step<1e-15):
            break
            
    return (df.set_index('Step'))





In [116]:
####### COORDINATE SEARCH ALGORITHM - OPPORTUNISTIC #######
"""
bb: Black box function, takes x0 as argument
x0: Tuple, inital point
step: Initial step distance
evalBdgt: Total evaluations budget
upBound: Upper boundary for all dimension of x0
lowBound: Lower boundary for all dimension of x0
"""   
def coodSearchOpp(bb, x0, step, evalBdgt, upBound, lowBound):
 
    initialEvBdgt = evalBdgt
    
    def checkOutOfBounds(x):
        return (x>upBound).sum()>0 or (x<lowBound).sum()>0
    
    if checkOutOfBounds(np.array(x0)):
        print("Initial points should be inside boundaries")
        return
    
    df = pd.DataFrame()
    cache = dict()
    minObj = bb(x0)
    cache[x0] = minObj
    evalBdgt = evalBdgt - 1
    
    #Main loop
    while(evalBdgt > 0):
        
        #Generating candidates
        points = np.zeros((len(x0)*2, len(x0)))
        values = np.full(len(x0)*2, np.inf)
        
        for k in range(len(x0)):
            points[k] = x0
            points[k,k] = points[k,k]+step
            points[len(x0)+k] = x0
            points[len(x0)+k, k] = points[len(x0)+k, k]-step
        
        #Evaluating candidates
        for i in range(len(points)):
            if(evalBdgt > 0):
                #failure case
                if(tuple(points[i]) in cache or checkOutOfBounds(points[i])):
                    step = step/2
                    break
                else:
                    values[i] = bb(points[i])
                    evalBdgt = evalBdgt-1
                    if(values[i] < minObj):
                        break
                    cache[tuple(points[i])] = values[i]
            else:
                break
        
        #Results
        minVal = np.min(values)
        
        if(minVal < minObj):
            #Updating final values
            minObj = minVal
            x0 = points[np.argmin(values)]
            
            #Creating new row for dataframe
            newRow = dict()
            newRow['Step'] = initialEvBdgt-evalBdgt
            for i in range(len(x0)):
                newRow[i]=x0[i]
            newRow['Obj'] = minObj
            df = df.append(newRow, ignore_index=True)
        
        #Stopping criterion
        if(step<1e-15):
            break
            
            
    return (df.set_index('Step'))


In [122]:
####### COORDINATE SEARCH ALGORITHM - DYNAMIC #######
"""
bb: Black box function, takes x0 as argument
x0: Tuple, inital point
step: Initial step distance
evalBdgt: Total evaluations budget 
upBound: Upper boundary for all dimension of x0
lowBound: Lower boundary for all dimension of x0
"""   
def coodSearchDyn(bb, x0, step, evalBdgt, upBound, lowBound):
 
    initialEvBdgt = evalBdgt
    bestDirection = 0
    
    def checkOutOfBounds(x):
        return (x>upBound).sum()>0 or (x<lowBound).sum()>0
    
    if checkOutOfBounds(np.array(x0)):
        print("Initial points should be inside boundaries")
        return
    
    df = pd.DataFrame()
    cache = dict()
    minObj = bb(x0)
    cache[x0] = minObj
    evalBdgt = evalBdgt - 1
    
    #Main loop
    while(evalBdgt > 0):
        
        #Generating candidates
        points = np.zeros((len(x0)*2, len(x0)))
        values = np.full(len(x0)*2, np.inf)
        
        for k in range(len(x0)):
            points[k] = x0
            points[k,k] = points[k,k]+step
            points[len(x0)+k] = x0
            points[len(x0)+k, k] = points[len(x0)+k, k]-step
        
        #Evaluating candidates
        for i in range(len(points)):
            if(evalBdgt > 0):
                #failure case
                if(tuple(points[bestDirection-i]) in cache or checkOutOfBounds(points[bestDirection-i])):
                    step = step/2
                    break
                else:
                    values[bestDirection-i] = bb(points[bestDirection-i])
                    evalBdgt = evalBdgt-1
                    if(values[bestDirection-i] < minObj):
                        bestDirection = (bestDirection-i) if (bestDirection-i)>=0 else (len(points)+bestDirection-i)
                        break
                    cache[tuple(points[bestDirection-i])] = values[bestDirection-i]
            else:
                break
        
        #Results
        minVal = np.min(values)
        
        if(minVal < minObj):
            #Updating final values
            minObj = minVal
            x0 = points[np.argmin(values)]
            
            #Creating new row for dataframe
            newRow = dict()
            newRow['Step'] = initialEvBdgt-evalBdgt
            for i in range(len(x0)):
                newRow[i]=x0[i]
            newRow['Obj'] = minObj
            df = df.append(newRow, ignore_index=True)
        
        #Stopping criterion
        if(step<1e-15):
            break
            
            
    return (df.set_index('Step'))


In [125]:
#Running Coordinate Search for all starting Points
for i in range(len(startPtsGRIEWANK)):
    df = coodSearch(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[i]),
                    step=100,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600)
    df.to_csv('/home/noyt/Desktop/BA6/MTH8418/Devoir2/executions/executions/GRIEWANK/{}/COOD_SEARCH_REGULAR/res.txt'.format(i+1), header=None, sep='\t')
    
    df = coodSearchOpp(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[i]),
                    step=100,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600)
    df.to_csv('/home/noyt/Desktop/BA6/MTH8418/Devoir2/executions/executions/GRIEWANK/{}/COOD_SEARCH_OPPORTUNISTIC/res.txt'.format(i+1), header=None, sep='\t')
    
    df = coodSearchDyn(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[i]),
                    step=100,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600)
    df.to_csv('/home/noyt/Desktop/BA6/MTH8418/Devoir2/executions/executions/GRIEWANK/{}/COOD_SEARCH_DYNAMIC/res.txt'.format(i+1), header=None, sep='\t')
    

In [124]:
print(coodSearch(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[4]),
                    step=100,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600))
print(coodSearchOpp(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[4]),
                    step=1000,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600))
print(coodSearchDyn(bb=bb_GRIEWANK, 
                    x0=tuple(startPtsGRIEWANK[4]),
                    step=100,
                    evalBdgt=2000,
                    upBound=600,
                    lowBound=-600))


                0           1        Obj
Step                                    
5.0   -286.400000  378.700000  56.720514
7.0   -186.400000  378.700000  45.171705
11.0  -186.400000  328.700000  37.197497
12.0  -136.400000  328.700000  32.918203
16.0  -136.400000  303.700000  28.821358
17.0  -111.400000  303.700000  27.215893
21.0  -111.400000  291.200000  25.318858
22.0   -98.900000  291.200000  24.652766
26.0   -98.900000  284.950000  23.799133
27.0   -92.650000  284.950000  23.469676
31.0   -92.650000  281.825000  22.996694
32.0   -89.525000  281.825000  22.862188
36.0   -89.525000  280.262500  22.650503
41.0   -90.306250  280.262500  22.001382
45.0   -90.696875  280.262500  21.805560
49.0   -90.696875  280.067187  21.754773
54.0   -90.794531  280.067187  21.725007
58.0   -90.843359  280.067187  21.713487
62.0   -90.843359  280.042773  21.708264
67.0   -90.855566  280.042773  21.705735
71.0   -90.855566  280.036670  21.704472
76.0   -90.855566  280.033618  21.703848
81.0   -90.85556